In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/Shareddrives/Unlimited/Colab/AlphaVision/"
MODEL = "densenet161_aug"
model_path = "models_" + MODEL + "/"

In [3]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/Shareddrives/Unlimited/Colab/AlphaVision/AlphaVision.zip" "data_2.zip"
# data_2.zip을 현재 디렉터리에 압축해제
!unzip "data_2.zip"

Archive:  data_2.zip
  inflating: dirty_mnist_2nd.zip     
  inflating: dirty_mnist_2nd_answer.csv  
  inflating: mnist_data.zip          
  inflating: sample_submission.csv   
  inflating: test_dirty_mnist_2nd.zip  


In [4]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

In [5]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
#from torchinfo import summary

from torchvision import transforms
from torchvision.models import densenet161

## 1. 커스텀 데이터셋 만들기

In [6]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

## 2. 이미지 어그멘테이션

In [7]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [8]:
trainset = MnistDataset('dirty_mnist', 'dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('test_dirty_mnist', 'sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=32, num_workers=8)
test_loader = DataLoader(testset, batch_size=32, num_workers=4)

## 3. ResNet50 모형

In [9]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Tue Feb 23 04:54:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = densenet161(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
#print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


## 4. 학습하기

In [11]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 10
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.71964, 0.53125
0: 0.69268, 0.54087
0: 0.69470, 0.52644
0: 0.69067, 0.53726
0: 0.68479, 0.53846
0: 0.69695, 0.53365
0: 0.68534, 0.55168
0: 0.69355, 0.53005
0: 0.69692, 0.52644
0: 0.69091, 0.53726
0: 0.68931, 0.54808
0: 0.68909, 0.52524
0: 0.69201, 0.52163
0: 0.69518, 0.53245
0: 0.69317, 0.54087
1: 0.68768, 0.54327
1: 0.68106, 0.54928
1: 0.69102, 0.52404
1: 0.69301, 0.53606
1: 0.68343, 0.54207
1: 0.69424, 0.54928
1: 0.68129, 0.55048
1: 0.68507, 0.53846
1: 0.68961, 0.54207
1: 0.68183, 0.54447
1: 0.68025, 0.56611
1: 0.67927, 0.54207
1: 0.67850, 0.53726
1: 0.67502, 0.55649
1: 0.67146, 0.56370
2: 0.66768, 0.56971
2: 0.65695, 0.57812
2: 0.68195, 0.55168
2: 0.66694, 0.57332
2: 0.66177, 0.58293
2: 0.65148, 0.56731
2: 0.65406, 0.59615
2: 0.66695, 0.56010
2: 0.65320, 0.57091
2: 0.65467, 0.58293
2: 0.64750, 0.59736
2: 0.64741, 0.59495
2: 0.66738, 0.57933
2: 0.64820, 0.59255
2: 0.63711, 0.59014
3: 0.64296, 0.60817
3: 0.60997, 0.63462
3: 0.65143, 0.59255
3: 0.63170, 0.62740
3: 0.63264, 0.61298


## 5. 추론하기

In [12]:
submit = pd.read_csv('sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv(path + model_path + "baseline_prediction.csv", index = False)